In [9]:
from IPython.core.debugger import prompt
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate, ChatPromptTemplate

load_dotenv()
chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}.")
template.format(country_a="Mexico", country_b="Thailand")



chat.predict(prompt)

'Hello! How can I assist you today?'

In [12]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geopraphy expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
    ]
)
prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
    
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')